In [1]:
import pandas as pd

In [2]:
import h2o

In [3]:
from h2o.automl import H2OAutoML

In [8]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,46 mins 20 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_mardoniofranca_noltt4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.811 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [9]:
df = h2o.import_file("data/dados_21.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
# Set response column as a factor
df['Target'] = df['Target'].asfactor()
response='Target'

# Split the dataset into train and test
train, test = df.split_frame(ratios = [.8], seed = 1234)

In [11]:
# Choose which columns to encode
encoded_columns = ["Order_Priority", "Ship_Mode", 
                   "Customer_Name", "Region", "Customer_Segment",
                   "Product_Category","Product_Sub-Category",
                   "Product_Container"]

In [13]:
# For k_fold strategy we need to provide fold column
fold_column = "kfold_column"
train[fold_column] = train.kfold_column(n_folds=5, seed=1234)

In [14]:
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [16]:
# Train a TE model
df_te = H2OTargetEncoderEstimator(fold_column=fold_column,
                                       data_leakage_handling="k_fold",
                                       blending=True,
                                       inflection_point=3,
                                       smoothing=10,
                                       noise=0.15,     # In general, the less data you have the more regularization you need
                                       seed=1234)

In [17]:
df_te.train(x=encoded_columns,
                 y=response,
                 training_frame=train)


targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%


original_names,encoded_column_names
Order_Priority,Order_Priority_te
Ship_Mode,Ship_Mode_te
Customer_Name,Customer_Name_te
Region,Region_te
Customer_Segment,Customer_Segment_te
Product_Category,Product_Category_te
Product_Sub-Category,Product_Sub-Category_te
Product_Container,Product_Container_te


In [18]:
train.head()

Order_Priority,Order_Quantity,Sales,Ship_Mode,Profit,Customer_Name,Region,Customer_Segment,Product_Category,Product_Sub-Category,Product_Container,Target,kfold_column
High,21,2781.82,Express Air,-695.26,Monica Federle,Nunavut,Corporate,Office Supplies,Storage & Organization,Large Box,0,2
High,35,3389.93,Express Air,737.94,Beth Paige,Northwest Territories,Consumer,Furniture,Office Furnishings,Large Box,1,0
Not Specified,7,2039.56,Express Air,-329.49,Bryan Davis,Northwest Territories,Corporate,Office Supplies,Storage & Organization,Large Box,0,0
Medium,24,1168.15,Express Air,-743.96,Muhammed MacIntyre,Northwest Territories,Small Business,Office Supplies,Storage & Organization,Large Box,0,1
Not Specified,45,237.28,Express Air,-2088.68,Bryan Mills,Northwest Territories,Small Business,Office Supplies,Appliances,Large Box,0,3
Critical,17,1368.14,Express Air,171.26,Fred Wasserman,Northwest Territories,Home Office,Office Supplies,Appliances,Large Box,1,3
Low,21,4429.69,Express Air,983.55,Filia McAdams,Atlantic,Small Business,Technology,Copiers and Fax,Large Box,1,4
Not Specified,40,19109.6,Express Air,-379.29,Sanjit Chand,West,Home Office,Technology,Copiers and Fax,Large Box,0,0
High,48,446.53,Express Air,-261.45,Rob Dowd,West,Corporate,Furniture,Office Furnishings,Large Box,0,3
Not Specified,27,2780.88,Express Air,595.38,Tony Chapman,West,Consumer,Furniture,Office Furnishings,Large Box,1,1


In [20]:
## New target encoded train and test sets
train_te = df_te.transform(frame=train, as_training=True)
test_te = df_te.transform(frame=test, noise=0)

In [21]:
train_te.head()

Order_Priority_te,Ship_Mode_te,Customer_Name_te,Region_te,Customer_Segment_te,Product_Category_te,Product_Sub-Category_te,Product_Container_te,Order_Priority,Ship_Mode,Customer_Name,Region,Customer_Segment,Product_Category,Product_Sub-Category,Product_Container,Order_Quantity,Sales,Profit,kfold_column,Target
0.559569,0.58041,0.659098,0.610872,0.583449,0.536438,0.295683,0.400156,High,Express Air,Monica Federle,Nunavut,Corporate,Office Supplies,Storage & Organization,Large Box,21,2781.82,-695.26,2,0
0.446459,0.471306,0.520634,0.421505,0.465287,0.481181,0.460577,0.280789,High,Express Air,Beth Paige,Northwest Territories,Consumer,Furniture,Office Furnishings,Large Box,35,3389.93,737.94,0,1
0.576438,0.5794,0.771075,0.529599,0.548548,0.513629,0.252692,0.388883,Not Specified,Express Air,Bryan Davis,Northwest Territories,Corporate,Office Supplies,Storage & Organization,Large Box,7,2039.56,-329.49,0,0
0.676452,0.677586,0.878011,0.676949,0.67473,0.62169,0.424415,0.468367,Medium,Express Air,Muhammed MacIntyre,Northwest Territories,Small Business,Office Supplies,Storage & Organization,Large Box,24,1168.15,-743.96,1,0
0.432187,0.41584,0.511986,0.407301,0.423128,0.343789,0.420648,0.251629,Not Specified,Express Air,Bryan Mills,Northwest Territories,Small Business,Office Supplies,Appliances,Large Box,45,237.28,-2088.68,3,0
0.414867,0.509595,0.511986,0.501056,0.475322,0.437544,0.514403,0.345384,Critical,Express Air,Fred Wasserman,Northwest Territories,Home Office,Office Supplies,Appliances,Large Box,17,1368.14,171.26,3,1
0.381733,0.406984,0.511986,0.413263,0.313322,0.471017,0.172726,0.203761,Low,Express Air,Filia McAdams,Atlantic,Small Business,Technology,Copiers and Fax,Large Box,21,4429.69,983.55,4,1
0.499392,0.502354,0.511986,0.484094,0.490747,0.564049,0.467456,0.311837,Not Specified,Express Air,Sanjit Chand,West,Home Office,Technology,Copiers and Fax,Large Box,40,19109.6,-379.29,0,0
0.452552,0.458781,0.427635,0.463037,0.453098,0.464202,0.452368,0.29457,High,Express Air,Rob Dowd,West,Corporate,Furniture,Office Furnishings,Large Box,48,446.53,-261.45,3,0
0.666035,0.658068,0.858494,0.630268,0.616822,0.600677,0.596632,0.448849,Not Specified,Express Air,Tony Chapman,West,Consumer,Furniture,Office Furnishings,Large Box,27,2780.88,595.38,1,1


In [26]:
y

'Target'

In [31]:
x_baseline = ['Order_Priority_te','Ship_Mode_te', 'Customer_Name_te',
              'Region_te', 'Customer_Segment_te', 'Product_Category_te',
               'Product_Sub-Category_te', 'Product_Container_te',
               'Order_Quantity','Sales','Target']


In [32]:
x = x_baseline
y = "Target"
x.remove(y)

In [33]:
# For binary classification, response should be a factor
train_te[y] = train_te[y].asfactor()
test_te[y] = test_te[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=40, seed=1)
aml.train(x=x, y=y, training_frame=test_te)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |
16:37:35.568: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 146.0.

███████████████████████████████████████████████████████████████| (done) 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DRF_1_AutoML_3_20240427_163734,0.899432,0.422634,0.836767,0.14678,0.361794,0.130895
XGBoost_lr_search_selection_AutoML_3_20240427_163734_select_grid_model_8,0.899242,0.43839,0.868262,0.173106,0.370557,0.137313
StackedEnsemble_BestOfFamily_1_AutoML_3_20240427_163734,0.893182,0.405132,0.839877,0.136932,0.353953,0.125283
GBM_grid_1_AutoML_3_20240427_163734_model_2,0.89072,0.402463,0.854983,0.143182,0.351284,0.123401
GBM_5_AutoML_3_20240427_163734,0.887689,0.422001,0.847194,0.186932,0.370486,0.13726
StackedEnsemble_AllModels_1_AutoML_3_20240427_163734,0.885417,0.424358,0.829175,0.159659,0.365618,0.133676
XRT_1_AutoML_3_20240427_163734,0.882102,0.44892,0.835305,0.168561,0.380815,0.14502
GBM_grid_1_AutoML_3_20240427_163734_model_3,0.857008,0.470058,0.841349,0.207955,0.392534,0.154083
XGBoost_grid_1_AutoML_3_20240427_163734_model_10,0.855871,0.512545,0.752777,0.198106,0.398012,0.158413
GBM_grid_1_AutoML_3_20240427_163734_model_7,0.854167,0.473796,0.808097,0.172159,0.390276,0.152315


In [34]:
aml.leader

Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_1_AutoML_3_20240427_163734


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    32                 32                          13025                  6            15           9.5           19            40            27.6562

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.1470296434554611
RMSE: 0.38344444637451863
LogLoss: 0.6693540562330539
Mean Per-Class Error: 0.20662878787878788
AUC: 0.8744318181818181
AUCPR: 0.8182291081497364
Gini: 0.7488636363636363

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3571428571428571
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      53   27   0.3375   (27.0/80.0)
1      5    61   0.0758   (5.0/66.0)
Total  58   88   0.2192   (32.0/146.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.357143     0.792208  47
max f2                       0.333333     0.868347  49
max f0point5                 0.625        0.827338  27
max accuracy                 0.625        0.815068  27
max precision                0.8          0.916667  12
max recall                   0.0769231    1         67
max specificity              1            0.9875    0
max absolute_mcc             0.625        0.630797  27
max min_per_class_accuracy   0.545455     0.787879  35
max mean_per_class_accuracy  0.6          0.807386  29
max tns                      1            79        0
max fns                      1            61        0
max fps                      0            80        70
max tps                      0.0769231    66        67
max tnr                      1            0.9875    0
max fnr                      1            0.924242  0
max fpr                      0            1         70
max tpr                      0.0769231    1         67

Gains/Lift Table: Avg response rate: 45,21 %, avg score: 45,70 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0410959                   1                  1.84343   1.84343            0.833333         1           0.833333                    1                   0.0757576       0.0757576                  84.3434   84.3434            0.0632576
2        0.0410959                   0.95               0         1.84343            0                0           0.833333                    1                   0               0.0757576                  -100      84.3434            0.0632576
3        0.0547945                   0.921474           2.21212   1.93561            1                0.930288    0.875                       0.982572            0.030303        0.106061                   121.212   93.5606            0.0935606
4        0.109589                    0.833333           1.93561   1.93561            0.875            0.861042    0.875                       0.921807            0.106061        0.212121                   93.5606   93.5606            0.187121
5        0.164384                    0.8                2.21212   2.02778            1                0.807273    0.916667                    0.883629            0.121212      

In [35]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
DRF_1_AutoML_3_20240427_163734,0.899432,0.422634,0.836767,0.14678,0.361794,0.130895,59,0.032886,DRF
XGBoost_lr_search_selection_AutoML_3_20240427_163734_select_grid_model_8,0.899242,0.43839,0.868262,0.173106,0.370557,0.137313,3410,0.045842,XGBoost
StackedEnsemble_BestOfFamily_1_AutoML_3_20240427_163734,0.893182,0.405132,0.839877,0.136932,0.353953,0.125283,3680,0.060762,StackedEnsemble
GBM_grid_1_AutoML_3_20240427_163734_model_2,0.89072,0.402463,0.854983,0.143182,0.351284,0.123401,86,0.033506,GBM
GBM_5_AutoML_3_20240427_163734,0.887689,0.422001,0.847194,0.186932,0.370486,0.13726,86,0.0576,GBM
StackedEnsemble_AllModels_1_AutoML_3_20240427_163734,0.885417,0.424358,0.829175,0.159659,0.365618,0.133676,3062,0.110749,StackedEnsemble
XRT_1_AutoML_3_20240427_163734,0.882102,0.44892,0.835305,0.168561,0.380815,0.14502,90,0.022894,DRF
GBM_grid_1_AutoML_3_20240427_163734_model_3,0.857008,0.470058,0.841349,0.207955,0.392534,0.154083,77,0.025842,GBM
XGBoost_grid_1_AutoML_3_20240427_163734_model_10,0.855871,0.512545,0.752777,0.198106,0.398012,0.158413,200,0.045408,XGBoost
GBM_grid_1_AutoML_3_20240427_163734_model_7,0.854167,0.473796,0.808097,0.172159,0.390276,0.152315,70,0.025606,GBM


https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html

https://medium.com/h2o-ai-brasil/h2o-os-primeiros-passos-fae39077e028

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/training-models.html